In [6]:
from utils import r14, r32, r50, r125, r134a, r143a, r170, atom_type, opt_atom_types
import numpy as np
import unyt as u
import pandas as pd
from fffit.fffit.utils import values_real_to_scaled, values_scaled_to_real, variances_scaled_to_real
from fffit.fffit.plot import plot_obj_contour

In [7]:
#Load class properies for each molecule
r14_class = r14.R14Constants()
r32_class = r32.R32Constants()
r50_class = r50.R50Constants()
r125_class = r125.R125Constants()
r134a_class = r134a.R134aConstants()
r143a_class = r143a.R143aConstants()
r170_class = r170.R170Constants()

In [ ]:
molec_data_dict = {"R14":r14_class, "R32":r32_class, "R50":r50_class, "R170":r170_class, "R125":r125_class, "R134a":r134a_class, "R143a":r143a_class}
df = pd.DataFrame(columns = ["Molecule", "Property", "Value", "Uncertainty", "Weight"])
for key in list(molec_data_dict.keys()):
    molec_object = molec_data_dict[key]
    for prop_key in list(molec_object.uncertainties.keys()):
        if "vap_density" in prop_key:
            exp_data = molec_object.expt_vap_density
            property_bounds = molec_object.vap_density_bounds
            property_name = "Vapor Density [kg/m^3]"
        elif "liq_density" in prop_key:
            exp_data = molec_object.expt_liq_density
            property_bounds = molec_object.liq_density_bounds
            property_name = "Liquid Density [kg/m^3]"
        elif "Pvap" in prop_key: 
            exp_data = molec_object.expt_Pvap
            property_bounds = molec_object.Pvap_bounds
            property_name = "Vapor pressure [bar]"
        elif "Hvap" in prop_key:
            exp_data = molec_object.expt_Hvap
            property_bounds = molec_object.Hvap_bounds
            property_name = "Enthalpy of Vaporization [kJ/kg]"

        values = np.array(list(exp_data.values()))
        y_unc = molec_object.uncertainties[prop_key]
        weight_unc = 1/(y_unc*values)**2
        weight_2pct = 1/(0.02*values)**2
        weight = np.maximum(weight_unc, weight_2pct)
        df_vals = [key, prop_key, values, y_unc, weight]
        iter_df = pd.DataFrame([df_vals], columns = df.columns)
        iter_df = iter_df.apply(lambda col: col.explode(), axis=0).reset_index(drop=True).copy(deep =True)
        df = pd.concat([df, iter_df], ignore_index=True)
# df.to_csv("molecule_exp_unc_data.csv", index = False, header = True)

In [9]:
repeats = 1
seed = 1
save_data = False
w_calc = 2
obj_choice = "ExpVal"
at_class = atom_type.AT_Scheme_9()
molec_data_dict = {"R134a":r134a_class}
pm = at_class.get_transformation_matrix(molec_data_dict)
print(pm)
theta_paper = np.array([3.727, 3.578558657, 3.754, 2.237, 2.607, 3.255, 2.982, 71.73, 11.25885226, 72.61, 23.13, 39.98, 20.164, 23.13])
t_paper = theta_paper.reshape(-1,1).T@pm
print(t_paper)

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]
[[ 3.754  3.754  2.237  2.607  2.982 72.61  72.61  23.13  39.98  23.13 ]]


In [25]:
import os
import tensorflow as tf
molec_data_dict = {"R14":r14_class, "R32":r32_class, "R50":r50_class, "R170":r170_class, "R125":r125_class, "R134a":r134a_class, "R143a":r143a_class}
all_gp_dict = opt_atom_types.get_gp_data_from_pkl(list(molec_data_dict.keys()))
visual = opt_atom_types.Vis_Results(molec_data_dict, all_gp_dict, at_class, w_calc, obj_choice, save_data)
for molec_names in list(molec_data_dict.keys()):
    param_matrix = visual.at_class.get_transformation_matrix({molec_names: molec_data_dict[molec_names]})
    #Get best_per_run.csv for all molecules
    molec_dir = visual.make_results_dir([molec_names])
    if os.path.exists(molec_dir+"/best_per_run.csv"):
        print(molec_dir)
        molec_df = pd.read_csv(molec_dir+"/best_per_run.csv", header = 0)
        # print(molec_df.head())
        first_param_name = at_class.at_names[0] + "_min"
        last_param_name = at_class.at_names[-1] + "_min"
        print(molec_df.loc[0, "Min Obj"])
        molec_best = molec_df.loc[0, first_param_name:last_param_name].values
        # ind_best_real = visual.values_pref_to_real(molec_best)
        # ind_best_scl = values_real_to_scaled(ind_best_real.reshape(1,-1), at_class.at_bounds_nm_kjmol)
        ind_best_gp = molec_best.reshape(-1,1).T@param_matrix
        # ind_best_gp = tf.convert_to_tensor(ind_best_gp, dtype=tf.float64)
        print(ind_best_gp)

Results/at_9/R14/ExpVal/wt_y_var
661.5452228409085
[[3.433271424285917 2.9202925438856786 30.567616530912 32.38449650118564]]
Results/at_9/R32/ExpVal/wt_y_var
8.08375707959312
[[3.8104463955181833 2.7068130354462 1.9855711710588373 48.7303136495141
  38.73076700200722 7.630628225268625]]
Results/at_9/R50/ExpVal/wt_y_var
30247.725059469245
[[3.75488555694032 2.4250966096926727 64.11703477015094 5.961044396934933]]
Results/at_9/R170/ExpVal/wt_y_var
141653.4403052184
[[3.6618972287220983 2.510684598364721 59.69006878343313
  5.139035267717284]]
Results/at_9/R125/ExpVal/wt_y_var
56454.88261823863
[[3.711020994853049 3.711020994853049 3.097250513695898 2.631982982251228
  1.966121438935412 36.150967527111526 36.150967527111526
  26.883926661781317 27.055062062674725 2.6542515622905576]]
Results/at_9/R134a/ExpVal/wt_y_var
891028.1282425477
[[3.247168942468984 3.247168942468984 2.9804977729861983
  2.899850039909928 2.608011778698622 60.80087284201044 60.80087284201044
  34.353261251818004 23

In [10]:
molec_data_dict = {"R134a":r134a_class}
all_gp_dict = opt_atom_types.get_gp_data_from_pkl(list(molec_data_dict.keys()))
at_optimizer =  opt_atom_types.Opt_ATs(molec_data_dict, all_gp_dict, at_class, repeats, seed, w_calc, obj_choice, save_data)
best_pref = np.array([3.727, 3.578558657, 3.754, 2.237, 2.607, 3.255, 2.982, 71.73, 11.25885226, 72.61, 23.13, 39.98, 20.164, 23.13])
best_real= at_optimizer.values_pref_to_real(best_pref)
obj = at_optimizer.calc_obj(best_real)
print(obj[0])
# jac, hess = at_optimizer.approx_jac_hess(best_real)
# print(scipy.linalg.norm(jac, ord = np.inf))

[[0.877      0.877      0.1185     0.3035     0.988      0.96323077
  0.96323077 0.23228571 0.71371429 2.64125    0.        ]
 [0.877      0.877      0.1185     0.3035     0.988      0.96323077
  0.96323077 0.23228571 0.71371429 2.64125    0.25      ]
 [0.877      0.877      0.1185     0.3035     0.988      0.96323077
  0.96323077 0.23228571 0.71371429 2.64125    0.5       ]
 [0.877      0.877      0.1185     0.3035     0.988      0.96323077
  0.96323077 0.23228571 0.71371429 2.64125    0.75      ]
 [0.877      0.877      0.1185     0.3035     0.988      0.96323077
  0.96323077 0.23228571 0.71371429 2.64125    1.        ]]
[[0.877      0.877      0.1185     0.3035     0.988      0.96323077
  0.96323077 0.23228571 0.71371429 2.64125    0.        ]
 [0.877      0.877      0.1185     0.3035     0.988      0.96323077
  0.96323077 0.23228571 0.71371429 2.64125    0.25      ]
 [0.877      0.877      0.1185     0.3035     0.988      0.96323077
  0.96323077 0.23228571 0.71371429 2.64125    0.5

In [4]:
molec_data_dict = {"R14":r14_class, "R32":r32_class, "R50":r50_class, "R170":r170_class, "R125":r125_class, "R134a":r134a_class, "R143a":r143a_class}
# molec_data_dict = {"R125":r125_class}
all_gp_dict = opt_atom_types.get_gp_data_from_pkl(list(molec_data_dict.keys()))
visual = opt_atom_types.Vis_Results(molec_data_dict, all_gp_dict, at_class, w_calc, obj_choice, save_data)
molec_names = list(["R14", "R32", "R50", "R170", "R125", "R134a", "R143a"])
visual.comp_paper_full_ind(molec_names)

[3.563847649565008 3.702329780457801 3.383454561616988 2.501869323173524
 2.8016532687774456 2.957056002591428 2.938081818350493 47.955100549476825
 42.9383601572982 51.7928754827407 9.956251656605064 39.956127381108494
 28.664773473001603 24.34860285294489]
[3.563847649565008 3.702329780457801 3.383454561616988 2.501869323173524
 2.8016532687774456 2.957056002591428 2.938081818350493 47.955100549476825
 42.9383601572982 51.7928754827407 9.956251656605064 39.956127381108494
 28.664773473001603 24.34860285294489]
[3.563847649565008 3.702329780457801 3.383454561616988 2.501869323173524
 2.8016532687774456 2.957056002591428 2.938081818350493 47.955100549476825
 42.9383601572982 51.7928754827407 9.956251656605064 39.956127381108494
 28.664773473001603 24.34860285294489]
[3.563847649565008 3.702329780457801 3.383454561616988 2.501869323173524
 2.8016532687774456 2.957056002591428 2.938081818350493 47.955100549476825
 42.9383601572982 51.7928754827407 9.956251656605064 39.956127381108494
 28

In [19]:
molec = "R32"
molec_data_dict = {molec:r32_class} #, "R32":r32_class, "R50":r50_class, "R170":r170_class, "R125":r125_class, "R134a":r134a_class, "R143a":r143a_class}
all_gp_dict = opt_atom_types.get_gp_data_from_pkl(list(molec_data_dict.keys()))
# all_gp_dict = {"R14" : {list(all_gp_dict["R14"].keys())[0]: list(all_gp_dict["R14"].values())[0]}}
# all_gp_dict = {molec : all_gp_dict[molec]}
at_optimizer =  opt_atom_types.Opt_ATs(molec_data_dict, all_gp_dict, at_class, repeats, seed, w_calc, obj_choice, save_data)
ls_results = at_optimizer.optimize_ats()
ls_results

KeyboardInterrupt: 

In [ ]:
# Determine the index of the row where column 'B' has the minimum value
min_index = ls_results['Min Obj Cum.'].idxmin()

# Retrieve the row corresponding to the minimum value in column 'B'
min_row = ls_results.iloc[min_index]
print(min_row)

In [ ]:
first_param_name = at_optimizer.at_class.at_names[0] + "_min"
last_param_name = at_optimizer.at_class.at_names[-1] + "_min"
molec_best = ls_results.loc[21, first_param_name:last_param_name].values
print(molec_best)

In [ ]:
#Get best values from csvs
def get_best_results(visual, molec_names):
    best_vals = {}
    #Get best_per_run.csv for all molecules
    all_molec_dir = visual.make_results_dir(molec_names)
    all_df = pd.read_csv(all_molec_dir+"/best_per_run.csv", header = 0)
    # print(all_df)
    first_param_name = visual.at_class.at_names[0] + "_min"
    last_param_name = visual.at_class.at_names[-1] + "_min"
    full_opt_best = all_df.loc[0, first_param_name:last_param_name].values
    best_vals["All"] = full_opt_best
    for i in range(len(molec_names)):
        molec_dir = visual.make_results_dir([molec_names[i]])
        molec_df = pd.read_csv(molec_dir+"/best_per_run.csv", header = 0)
        molec_best = molec_df.loc[0, first_param_name:last_param_name].values
        best_vals[molec_names[i]] = molec_best
    return best_vals

In [ ]:
molec_data_dict = {"R14":r14_class, "R32":r32_class, "R50":r50_class, "R170":r170_class, "R125":r125_class, "R134a":r134a_class, "R143a":r143a_class}
all_gp_dict = opt_atom_types.get_gp_data_from_pkl(list(molec_data_dict.keys()))
at_optimizer =  opt_atom_types.Opt_ATs(molec_data_dict, all_gp_dict, at_class, repeats, seed, False, save_data)
best_pref = np.array([3.002,	4.000,	1.500,	3.231,	3.394,	2.927,	38.333,	75.000,	10.000,	50.000,	50.000,	36.598])
# best_pref = ls_results.loc[121, "sigma_C1_min":"epsilon_F_Hx_min"].to_numpy()
# best_real = at_optimizer.values_pref_to_real(best_pref)
best_real= at_optimizer.values_pref_to_real(best_pref)
obj = at_optimizer.calc_obj(best_real)
print(obj[0])
jac, hess = at_optimizer.approx_jac_hess(best_real)
print(scipy.linalg.norm(jac, ord = np.inf))

In [ ]:
molec_data_dict = {"R32":r32_class}#, "R32":r32_class, "R50":r50_class, "R170":r170_class, "R125":r125_class, "R134a":r134a_class, "R143a":r143a_class}
all_gp_dict = opt_atom_types.get_gp_data_from_pkl(list(molec_data_dict.keys()))

visual = opt_atom_types.Vis_Results(molec_data_dict, all_gp_dict, at_class, scl_w, save_data)
best_set = best_pref
visual.plot_obj_hms(best_set)